# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119410e+02     1.592904e+00
 * time: 0.14232492446899414
     1     1.167310e+01     1.063397e+00
 * time: 1.5224330425262451
     2    -1.102771e+01     1.230674e+00
 * time: 1.6672170162200928
     3    -3.323375e+01     8.630808e-01
 * time: 1.8418309688568115
     4    -4.658742e+01     6.434574e-01
 * time: 2.0116188526153564
     5    -5.668375e+01     2.333021e-01
 * time: 2.1648318767547607
     6    -5.967556e+01     1.902306e-01
 * time: 2.297433853149414
     7    -6.091509e+01     6.026997e-02
 * time: 2.4313628673553467
     8    -6.140307e+01     3.716817e-02
 * time: 2.560694932937622
     9    -6.170865e+01     3.225764e-02
 * time: 2.6791598796844482
    10    -6.185799e+01     3.267211e-02
 * time: 2.8096139430999756
    11    -6.200633e+01     2.196141e-02
 * time: 2.9330968856811523
    12    -6.208368e+01     1.388308e-02
 * time: 3.0668790340423584
    13    -6.212832e+01     1.243505e-02
 * time: 3.18251585960

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671068
    AtomicLocal         -18.8557681
    AtomicNonlocal      14.8522648
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485372 
    Xc                  -19.3336820

    total               -62.261666455028
